In [1]:
import requests
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

In [5]:
# An example query using the api
'''
api_url = "https://imslp.org/api.php?title=Category:For_piano"

info_params = {
    "action": "query",
    "format": "json",
    "list": "alllinks",
    "alnamespace": "0",
    "alunique": "1"
}

response = requests.get(api_url, params=info_params)
info_data = response.json()

print(info_data)

# construct a list of urls to scrape
urls = []
for entry in info_data['query']['alllinks']:
    url = 'https://imslp.org/wiki/' + entry['title']
    url = re.sub(r'\s', '_', url)
    urls.append(url)
print(urls)
'''


{'query': {'alllinks': [{'ns': 0, 'title': '!shorter!longer!shorter! (Fuchs, Armin)'}, {'ns': 0, 'title': '"Ah! Perfido" (Scene and Aria), Op.65 (Beethoven, Ludwig van)'}, {'ns': 0, 'title': '"Air" from Orchestral Suite No. 3, BWV 1068'}, {'ns': 0, 'title': '"Et nomen domini" per soprano, archi e bc (Massimi, Francesco)'}, {'ns': 0, 'title': '"My Old Dutch" Waltz (West, Alfred H.)'}, {'ns': 0, 'title': '"So he passed over, and all the Trumpets sounded for him on the other side" (Vaughan Williams, Ralph)'}, {'ns': 0, 'title': '"Twinkle, Twinkle, Little Star" Variations (Suzuki, Shinichi)'}, {'ns': 0, 'title': '"Und ich sah ...", Op.59 (Salbert, Dieter)'}, {'ns': 0, 'title': '$1'}, {'ns': 0, 'title': "'''Musical Masterpiece'''"}]}, 'query-continue': {'alllinks': {'alfrom': "'''http://www.caballero-ntra-sra.com.ar/0/6/0/0.htm'''"}}}
['https://imslp.org/wiki/!shorter!longer!shorter!_(Fuchs,_Armin)', 'https://imslp.org/wiki/"Ah!_Perfido"_(Scene_and_Aria),_Op.65_(Beethoven,_Ludwig_van)', 'ht

In [25]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://imslp.org/index.php?title=Category:For_piano"

# Send a GET request to the webpage
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find all anchor tags (links) in the webpage
links = soup.find_all('a')

# Extract the href attribute from each link
link_list = [link.get('href') for link in links]

# Filter out None values and remove empty strings
link_list = [link for link in link_list if link is not None and link.strip()]
urls = link_list[81:81+80]
urls= ['https://imslp.org' + s for s in urls]
print(urls)

['https://imslp.org/wiki/%22Mrs._Ramsay_rose._Lily_rose.%22_(Eisenbrey,_Keith)', 'https://imslp.org/wiki/%22of_being_a_self_in_a_song%22_(Feng,_Thomas)', 'https://imslp.org/wiki/%27Anna_Bolena%27_fantaisie,_Op.204_(Leduc,_Alphonse)', 'https://imslp.org/wiki/%27Do_love_me%27_Waltz,_Op.332_(Kontski,_Anton_de)', 'https://imslp.org/wiki/The_%27Five_Pound%27_Polka_(Glover,_Stephen_Ralph)', 'https://imslp.org/wiki/%27Gae%27_Minuet,_WoO_3_(Schreiber,_Robin)', 'https://imslp.org/wiki/%27Leven_Forty-Five_From_The_Hotel_(Mills,_Kerry)', 'https://imslp.org/wiki/%27n_Avond_Blomkens_(Tinel,_Jef)', 'https://imslp.org/wiki/%27Neath_Hawaiian_Skies_(Rawlings,_Charles_Arthur)', 'https://imslp.org/wiki/%27O_Malho%27_na_ponta_(Guimar%C3%A3es,_Amadeu)', 'https://imslp.org/wiki/%27O_Malho%27_(Bastos,_Henrique_Teixeira_de_Souza)', 'https://imslp.org/wiki/%27O_Malho%27_(Braga,_Jos%C3%A9_Mauricio)', 'https://imslp.org/wiki/%27O_quanto_l%27anima%27_with_a_Rondo_for_Piano_4-Hands_(Jansen,_Louis)', 'https://imslp

In [ ]:
# code to get page content in HTML

url = 'https://imslp.org/wiki/Andante_spianato_et_Grande_polonaise_brillante,_Op.22_(Chopin,_Fr%C3%A9d%C3%A9ric)'
response = requests.get(url)
response.raise_for_status()
page_content = response.text
print(page_content)

In [9]:
# functions to get page text and parse out the general information sectioin

def get_page_text(url):
    # parse web content into English text
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors

        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page, excluding script and style tags
        text = soup.get_text(separator='\n', strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def extract_text_between_strings_second(text, start_string, end_string):
    # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
    if text:
        start_index = text.find(start_string, text.find(start_string) + len(start_string))
        if start_index == -1:  # If start_string is not found
            return None

        # Find the occurrence of the end_string
        end_index = text.find(end_string, start_index)
        if end_index == -1:  # If end_string is not found
            return None

        # Extract the substring between the second occurrence of start_string and end_string
        extracted_text = text[start_index + len(start_string):end_index].strip()

        return extracted_text

def retrieve_general_information(url):
    # retrieve the general information section
    parsed_text = extract_text_between_strings_second(get_page_text(url),'General Information','Instrumentation')
    return parsed_text

In [26]:
# an example parsing

url_to_parse = ['https://imslp.org/wiki/Ballade_No.3,_Op.47_(Chopin,_Fr%C3%A9d%C3%A9ric)',
                'https://imslp.org/wiki/%C3%89tudes,_Op.10_(Chopin,_Fr%C3%A9d%C3%A9ric)',
                'https://imslp.org/wiki/Piano_Sonata_No.2%2C_Op.36_(Rachmaninoff%2C_Sergei)',
                'https://imslp.org/wiki/Barcarolle_No.11%2C_Op.105_(Faur%C3%A9%2C_Gabriel)',
                'https://imslp.org/wiki/English_Suite_No.1_in_A_major%2C_BWV_806_(Bach%2C_Johann_Sebastian)'] + urls

def extract_text_between_strings_new(text, start_string, end_string):
            # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
            start_index = text.find(start_string)
            if start_index == -1:  # If start_string is not found
                return None

            # Find the occurrence of the end_string
            end_index = text.find(end_string, start_index)
            if end_index == -1:  # If end_string is not found
                return None

            # Extract the substring between the second occurrence of start_string and end_string
            extracted_text = text[start_index + len(start_string):end_index].strip()

            return extracted_text
        
attribute_list = ['Work Title', 'Alt\nernative\n.\nTitle', 'Name Translations', 'Name Aliases', 'Authorities', 'Composer',
                        'Opus/Catalogue Number', 'I-Catalogue Number', 'Key', 'Movements/Sections', 'Year/Date of Composition', 
                        'First Pub\nlication', 'Dedication', 'Average Duration', 'Composer Time Period', "Piece Style"]
list_of_deletable_headings = ['Opus/Catalogue Number','I-Catalogue Number','Movements/Sections','Year/Date of Composition','First Pub\nlication','Average Duration','Composer Time Period']
bad_heading_pattern = r'^.*\n'

info = []
for url in url_to_parse:
    parsed_text = retrieve_general_information(url)
    
    if parsed_text:

        info_dic = {}
        
        present_attributes = []
        # check which attributes are present in the parsed text
        for attribute in attribute_list:
            if parsed_text.find(attribute) != -1:
                present_attributes.append(attribute)
        
        # extract the text for each present attribute
        for attribute in attribute_list:
            if attribute in present_attributes:
                info_dic[attribute] = extract_text_between_strings_new(parsed_text, attribute, present_attributes[present_attributes.index(attribute) + 1] if present_attributes.index(attribute) != len(present_attributes) - 1 else "Romantic")
                
            else:
                info_dic[attribute] = None
                
        info_dic['Piece Style'] = parsed_text[parsed_text.find('Piece Style')+12:] # extract the piece style from the end of the general information section
        info_dic['url'] = url
        for heading in list_of_deletable_headings:
            if info_dic[heading]:
                info_dic[heading] = re.sub(bad_heading_pattern, '', info_dic[heading])
        info.append(info_dic)
        
    else:
        print("String not found or invalid input.")
        
# convert the dictionary to a pandas dataframe
df_info = pd.DataFrame(info)
del df_info['Alt\nernative\n.\nTitle'] # drop unncecessary column
df_info.rename(columns={'First Pub\nlication': 'First Publication'}, inplace=True) # rename column
print(df_info.head())

           Work Title                                  Name Translations  \
0        Ballade No.3  Ballade No. 3 in A-flat major, Op. 47\n;\nBall...   
1              Etudes  Étude Op. 10, No. 10\n;\n연습곡 10번\n;\nEtiuda op...   
2   Piano Sonata No.2  第2号钢琴奏鸣曲\n;\n第2號鋼琴奏鳴曲\n;\nピアノソナタ第2番\n;\nKlavir...   
3    Barcarolle No.11                                               None   
4  English Suite No.1                                               None   

                                        Name Aliases  \
0                            Balada n.º 3\n;\nOp. 47   
1                                               None   
2  降B小調第二號鋼琴奏鳴曲\n;\nSergej Rahmaninov: Klavirska ...   
3                                               None   
4                                               None   

                                         Authorities                Composer  \
0  WorldCat\n;\nWikipedia\n;\nVIAF\n:\n301552296\...        Chopin, Frédéric   
1  WorldCat\n;\nWikipedia\n;\nVIAF\n:\n1760059

In [29]:
# examine the dataframe
print(df_info.info())

# save the dataframe to a csv file
df_info.to_csv('imslp_piano_music.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Work Title                85 non-null     object
 1   Name Translations         4 non-null      object
 2   Name Aliases              3 non-null      object
 3   Authorities               4 non-null      object
 4   Composer                  85 non-null     object
 5   Opus/Catalogue Number     19 non-null     object
 6   I-Catalogue Number        85 non-null     object
 7   Key                       52 non-null     object
 8   Movements/Sections        29 non-null     object
 9   Year/Date of Composition  45 non-null     object
 10  First Publication         52 non-null     object
 11  Dedication                35 non-null     object
 12  Average Duration          23 non-null     object
 13  Composer Time Period      85 non-null     object
 14  Piece Style               85